In [ ]:
def allsymbols(path):
    import MetaTrader5 as mt5
    if not mt5.initialize(path):
        print("initialize() failed, error code =",mt5.last_error())
        quit()

    allsymbol=[]
    symbols=mt5.symbols_get()
    count=0
    for s in symbols:
        count+=1
        allsymbol.append(s.name)
    return allsymbol

def readdata(market,tf,barre,path):
    from datetime import datetime
    import MetaTrader5 as mt5
    import pandas as pd

    if not mt5.initialize(path):
        print("initialize() failed, error code =",mt5.last_error())
        quit()
    
    mk=str(market)
    TF="mt5.TIMEFRAME_"+str(tf)
    bar=barre
     
    while True:

        try:
            rates = mt5.copy_rates_from_pos(mk, eval(TF), 0, bar) #1700
            if len(rates)>0:
                break
        except:
            continue

    rates_frame = pd.DataFrame(rates)
    rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
    rates_frame=rates_frame[["time","open","high","low","close","tick_volume"]]
    rates_frame.columns=(["time","open","high","low","close","volume"])
    rates_frame=rates_frame.set_index(["time"])
    rates_frame=rates_frame[(rates_frame.T != 0).any()]
    return rates_frame

def get_poin_values(path,simb):

    import MetaTrader5 as mt5
    if not mt5.initialize(path):
        print("initialize() failed, error code =",mt5.last_error())
        quit()
    symbol_info=mt5.symbol_info(simb)
    if symbol_info!=None:
        symbol_info_dict = mt5.symbol_info(simb)._asdict()
    mt5.shutdown()    
    tmp=symbol_info_dict['trade_tick_value']/symbol_info_dict['point']
    return tmp

def readdata_MT5_Date(market,tf,path,start,stop):
    from datetime import datetime
    import MetaTrader5 as mt5
    import pandas as pd
    
    while True:

        if not mt5.initialize(path):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        mk=str(market)
        TF="mt5.TIMEFRAME_"+str(tf)

        start_y , start_m , start_d = start.split("-")
        stop_y , stop_m , stop_d = stop.split("-")
        
        utc_from = datetime(int(start_y), int(start_m), int(start_d))
        utc_to = datetime(int(stop_y), int(stop_m), int(stop_d))
        
        rates = mt5.copy_rates_range(mk, eval(TF), utc_from, utc_to) 
    

        rates_frame = pd.DataFrame(rates)
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame=rates_frame[["time","open","high","low","close","tick_volume"]]
        rates_frame.columns=(["time","open","high","low","close","volume"])
        rates_frame=rates_frame.set_index(["time"])
        #rates_frame=rates_frame[(rates_frame.T != 0).any()]
        if rates_frame is not None:
            return rates_frame
            break

In [ ]:
def rnd_gene():
    """
    Funzione che attiva un gene casuale
    cioè un indice di colonna nella tabella rules.
    """
    return np.random.randint(0, len(rules.columns) - 1)


def rnd_dna():
    """
    Funzione che genera un dna casuale (a singolo cromosoma, solo regole in "and" logico).
    """
    dna = []
    for i in range(0, DNA_SIZE):
        dna.append(rnd_gene())
    return dna


def first_generation():
    """
    Funzione che genera un dataframe con la popolazione iniziale casuale. 
    Ogni riga corrisponde a un DNA cioè ad un investitore artificiale.
    """
    population = pd.DataFrame(index = range(POPULATION_SIZE), columns = range(DNA_SIZE))
    for i in range(0, POPULATION_SIZE):
        population.iloc[i] = rnd_dna()
    return population


def dna_formula(dna):
    """
    Funzione che calcola la formula di un DNA (a singolo cromosoma).
    """
    formula = ""
    for gene in dna:
        if len(formula) != 0:
            formula += " & " ## "and"
        formula += "("+rule_formulas[gene]+")"
    return formula


@numba.jit(nopython = True) # ACCELARAZIONE NUMBA: aggiungo un'annotazione per compilare il metodo setups() con Numba
def setups(dna):
    """
    Funzione che calcola i setups di un dna (ad unico cromosoma) su tutte le regole
    Versione che sostituisce la precedente funzione in Pandas
    """
    setups = None
    for gene in dna:
        if setups is None:
            setups = rules_array[gene] # ACCELARAZIONE NUMPY: usiamo l'array al posto del dataframe
        else:
            setups = setups & rules_array[gene] # ACCELARAZIONE NUMPY: usiamo l'array al posto del dataframe
    return setups


def plot_equity(dna):
    """
    Funzione che grafichi l'equity line risultatnte 
    dato un DNA.
    """
    gains = all_gains(dna)
    times, gains = zip(*gains)
    profits = np.array(gains).cumsum()
    equity = pd.Series(profits, times)
    equity.plot(figsize=(14, 8), grid = True)

    
def sort_population(population):
    """
    Funzione per operare un ordinamento della popolazione
    rispetto alla fitness function
    """
    scores = population.apply(lambda dna: fitness(dna), axis = 1)
    sorted_population = population.copy()
    sorted_population["Score"] = scores
    sorted_population.sort_values(by = ["Score"], ascending = [False], inplace = True)
    sorted_population.reset_index(drop = True, inplace = True)
    sorted_population.drop(sorted_population.columns[[-1,]], axis = 1, inplace = True)
    return sorted_population


def mutation(mut_prob,dna1):
    """
    Funzione che muta un DNA cambiando ogni gene con un gene casuale
    con probabilità MUTATION_PROB.
    """
    dna = []
    for i in range(DNA_SIZE):
        if np.random.random() < mut_prob:
            dna.append(rnd_gene())
        else:
            dna.append(dna1[i])
    return dna


def crossover(dna1, dna2):
    """
    Funzione che, presi due DNA, in modo randomico, 
    prende il gene di uno dei due genitori
    """
    dna = []
    for i in range(DNA_SIZE):
        if np.random.randint(2) == 0:
            dna.append(dna1[i])
        else:
            dna.append(dna2[i])
    return dna

In [ ]:
def stack_sort(gains,secondary_equity):
    c=np.vstack((gains,secondary_equity))  
    d=sorted(c,key=lambda x: x[0])
    return d

def STD(dna):
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    
    if (ops >= MIN_OPERATIONS)&(sum([gain for (time, gain) in gains])>0):  
        gains=stack_sort(gains,secondary_equity)  
        times, gains = zip(*gains)
        return np.std(np.cumsum(gains))
    else:
        return -9999999999999999999

def profit(dna):
    """
    Fitness Function basata sul Profit totale.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    
    if ops >= MIN_OPERATIONS:  
        gains=stack_sort(gains,secondary_equity)   
        return round(sum([gain for (time, gain) in gains]),2)
    else:
        return 0
    
def kestner(dna):
    from scipy import stats    
    """
    Fitness Function basata sul Kestner Ratio.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    
    if ops >= MIN_OPERATIONS:  
        gains=stack_sort(gains,secondary_equity)  
        times, gains = zip(*gains)
                
        index = np.array(np.arange(1,len(times) + 1))

        x = index
        y = np.cumsum(gains)
        gradient, intercept, r_value, p_value, std_err = stats.linregress(x,y)

        if std_err != 0 and len(index) > 0:
            return round(gradient / (std_err * len(index)),5)
        else:
            return -999999
    else:
        return -999999    
    
def max_drawdown(dna):
    """
    Fitness Function basata sul Draw Down.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:
        gains=stack_sort(gains,secondary_equity)   
        eq = np.cumsum([i[1] for i in gains])

        eq_top_list = []
        eq_top = 0
        for el in eq:
            if el > eq_top:
                eq_top = el
            eq_top_list.append(eq_top)

        #print(len(eq),len(eq_top_list))

        dd = eq - eq_top_list

        if ops >= MIN_OPERATIONS and len(dd) != 0:

            return min(dd)
        else:
            return -np.inf
    else:
        return -np.inf
    
    
def avg_trade(dna):
    """
    Fitness Function basata sul AvgTrade.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:
        gains=stack_sort(gains,secondary_equity)   
        my_sum = sum([gain for (time, gain) in gains])

        if ops == 0 or ops < MIN_OPERATIONS:
            return 0
        else:
            return my_sum / ops
    else:
        return 0
    
    
def gross_profit(dna):
    """
    Fitness Function basata sul Gross Profit.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:
        gains=stack_sort(gains,secondary_equity)   
        pos_sum = sum([gain for (time, gain) in gains if gain > 0])

        if ops < MIN_OPERATIONS:
            return 1
        else:
            return pos_sum
    else:
        return 1
    
def profit_factor(dna):
    """
    Fitness Function basata sul Profit Factor.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:
        gains=stack_sort(gains,secondary_equity)   
        pos_sum = sum([gain for (time, gain) in gains if gain > 0])
        neg_sum = sum([gain for (time, gain) in gains if gain < 0])
        #print(pos_sum,neg_sum)

        if neg_sum == 0 or ops < MIN_OPERATIONS:
            return 0
        else:
            return round(abs(pos_sum / neg_sum),2)
    else:
        return 0
    
def SQN(dna):
    """
    Fitness Function basata sul SQN.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])    
    if ops >= MIN_OPERATIONS:
        gains=stack_sort(gains,secondary_equity)
        my_sum = np.sum([i[1] for i in gains])
        my_std = np.std([gain for (time, gain) in gains])
        sqn = np.sqrt(len(gains))*(my_sum/len(gains))/ my_std  
        return sqn
    else:
        return 0
    
def fitness(dna):
    """
    Funzione che associa la funzione di fitness
    su cui operare l'ordinamento al termine di ciascuna generazione
    """
    if FITNESS_FUNCTION == "Profit":
        return profit(dna)
    if FITNESS_FUNCTION == "AvgTrade":
        return avg_trade(dna)
    if FITNESS_FUNCTION == "ProfitFactor":
        return profit_factor(dna)
    if FITNESS_FUNCTION == "GrossProfit":
        return gross_profit(dna)
    if FITNESS_FUNCTION == "Profit/MaxDrawDown":
        return profit(dna) / -max_drawdown(dna)
    if FITNESS_FUNCTION == "AvgTrade/MaxDrawDown":
        return avg_trade(dna) / -max_drawdown(dna)
    if FITNESS_FUNCTION == "KestnerRatio":
        return kestner(dna)
    if FITNESS_FUNCTION == "StandardDeviation":
        return STD(dna)
    if FITNESS_FUNCTION == "SQN":
        return SQN(dna)

In [ ]:
def next_generation(population):
    """
    Funzione che riporta la generazione successiva
    """
    n = len(population.index)
    n_best = int(n * BEST_DNA_RATIO)
    n_cross = int(n * CROSS_DNA_RATIO)
    n_mutate = n - n_best - n_cross

    new_population = pd.DataFrame(index = range(POPULATION_SIZE), columns = range(DNA_SIZE))

    # copia i migliori DNA
    for i in range(n_best):
        new_population.iloc[i] = population.iloc[i]

    # aggiunge gli incroci
    for i in range(n_cross):
        dna1 = population.iloc[np.random.randint(n_best)]
        dna2 = population.iloc[np.random.randint(n_best)]
        dna = crossover(dna1, dna2)
        new_population.iloc[n_best + i] = dna

    # aggiunge le mutazioni
    for i in range(n_mutate):
        dna1 = population.iloc[np.random.randint(n_best)]
        dna = mutation(MUTATION_PROB, dna1)
        new_population.iloc[n_best + n_cross + i] = dna

    return new_population


def print_best(population, gen_num):
    """
    Funzione per la stampa delle performance del migliore investitore/DNA
    di una determinata generazione.
    """
    best_dna = population.iloc[0]    
    #tf = total_profit(best_dna)
    global tf
    tf = fitness(best_dna)
    #print("generation", gen_num, ":", round(tf, 2), " ", dna_formula(best_dna))
    return "generation", gen_num, ":", round(tf, 2), " ", dna_formula(best_dna) 

from IPython.display import clear_output
def evolution():
    """
    Funzione che definisce il ciclo principale dell'algoritmo genetico
    """
    best_dnas = []
    global log
    log = []
    population = first_generation()
    population = sort_population(population)
    best_dnas.append(population.iloc[0].values)
    
    
    # evolve for NUM_GENERATIONS generations
    for i in range(1, NUM_GENERATIONS):
        population = next_generation(population)
        population = sort_population(population)
        best_dnas.append(population.iloc[0].values)
        
        #print_best(population, i)
        clear_output(wait=True)
        print(print_best(population, i))
        log.append(tf)
        plot_equity(population.iloc[0])
        plt.legend()
        plot_equity_secondary(secondary_equity)
        plt.legend()
        plt.show()
        plot_equity_full(population.iloc[0],secondary_equity)
        plt.title("Aggregate Strategies")
        plt.show()
    return best_dnas

In [ ]:
def plot_equity(dna):
    """
    Funzione che grafichi l'equity line risultatnte 
    dato un DNA.
    """
    gains = all_gains(dna)
    if len(gains)>0:
        times, gains = zip(*gains)
        profits = np.array(gains).cumsum()
        equity = pd.Series(profits, times)
        equity.plot(figsize=(14, 8), grid = True, label="main strategies")

def plot_equity_secondary(secondary_equity):
    """
    Funzione che grafichi l'equity line risultatnte 
    dato un DNA.
    """
    if len(secondary_equity)>2:
        gains = secondary_equity
        times, gains = zip(*gains)
        profits = np.array(gains).cumsum()
        equity = pd.Series(profits, times)
        equity.plot(figsize=(14, 8), grid = True, label="secondary strategies")
        
def plot_equity_full(dna,secondary_equity):
    """
    Funzione che grafichi l'equity line risultatnte 
    dato un DNA.
    """
    gains = all_gains(dna)
    if len(gains)>0:
        gains=gains=stack_sort(gains,secondary_equity)
        times, gains = zip(*gains)
        profits = np.array(gains).cumsum()
        equity = pd.Series(profits, times)
        equity.plot(figsize=(14, 8), grid = True, color="green")

In [ ]:
from numba import jit
@jit(nopython=True)
def numba_engine(rule,TIME_EXIT,target_monetario,COSTS_FIXED):
    
    for el in range(len(rule)):
        if rule[el] == 1:
            rule[el+1:el+TIME_EXIT+1]=0
    d=rule*target_monetario
    d[d != 0 ]=d[d != 0 ]-(COSTS_FIXED*2)
    return d

def target(DIRECTION,TIME_EXIT,dataset):
    price=dataset.open.copy()
    t_m=[]
    t_m=price.shift(-TIME_EXIT)-price
    t_m=t_m.shift(-1)
    t_m=t_m.fillna(0)
    if DIRECTION == "long":
        t_m=np.array(t_m * BIGPOINTVALUE)
    else:
        t_m=t_m*(-1)
        t_m=np.array(t_m * BIGPOINTVALUE)
    return t_m

def all_gains(dna):
    
    rule = setups(numba.typed.List(dna))
    rule.setflags(write=1)
    rule=rule[:Out_Of_Sample_pct]
    gains = numba_engine(rule,TIME_EXIT,gains_array,COSTS_FIXED)
    tmp_times_array = times_array
    ls=[]
    for e in range(len(gains)):
        if gains[e]!=0:
            ls.append([tmp_times_array[e],gains[e]])
    return ls

def OOS(dataset,pct):
    if pct == 0:
        return len(dataset)
    else:
        return int(len(dataset) - (len(dataset) // (100/pct)))
    
def instability(array,pct):
    dimension = int(len(array) - (len(array) // (100/pct)))
    rng = np.random.default_rng()
    rand_array=rng.choice(array, dimension)
    return np.array(sorted(sorted(rand_array,key=lambda e:e[1]),key=lambda e:e[0]))

def check_secondary_equity(secondary_equity):
    if len(secondary_equity)>1:
        return secondary_equity
    else:
        return [np.datetime64(start_test+'T16:00:00.000000000'),0]    